# Cuaderno 01: Generación del Dataset Auditivo

**Objetivo:** Crear un dataset de archivos de audio `.wav` a partir de una lista de fonemas/letras clave para español e inglés.

**Flujo de Trabajo:**
1.  **Instalación de Librerías**: Instalar `gTTS` para la síntesis de voz y `pydub` para la conversión de audio.
2.  **Definición de Fonemas**: Crear las listas de sonidos a generar para cada idioma.
3.  **Generación de Audio**: Iterar sobre las listas, generar un archivo `.mp3` para cada fonema usando `gTTS`, y convertirlo al formato `.wav` usando `pydub`.

## Paso 1: Instalación de Librerías

Instalamos las dos librerías principales que necesitamos para este cuaderno.
* `gTTS` (Google Text-to-Speech): Convierte texto en audio.
* `pydub`: Una librería de alto nivel para manipulación de audio, que usaremos para exportar a formato `.wav`.

In [1]:
%pip install gTTS pydub

Note: you may need to restart the kernel to use updated packages.


## Paso 2: Configuración y Definición de Listas de Fonemas

Importamos las librerías necesarias, definimos las rutas de salida y creamos las listas de fonemas/letras para cada idioma. Las listas están diseñadas para cubrir los sonidos más representativos.

In [2]:
import os
from pathlib import Path
from gtts import gTTS
from pydub import AudioSegment

# --- Rutas de Salida ---
project_root = Path.cwd().parent
# Creamos carpetas de salida específicas para cada idioma
output_dir_es = project_root / "data/02_processed/phoneme_audio/es"
output_dir_en = project_root / "data/02_processed/phoneme_audio/en"

output_dir_es.mkdir(parents=True, exist_ok=True)
output_dir_en.mkdir(parents=True, exist_ok=True)


# --- Listas de Fonemas/Letras ---

# Lista para español (incluye vocales, consonantes y dígrafos)
phonemes_es = [
    'a', 'e', 'i', 'o', 'u',
    'b', "c", 'd', 'f', 'g', 'j',"h", 'k', 'l', 'm', 'n', 'ñ', 'p', 'r', 'rr', 's', 't', 'y',"v", 'z',
    'ch', 'll'
]

# Lista para inglés (incluye sonidos de vocales cortas/largas y dígrafos comunes)
phonemes_en = [
    'a', 'e', 'i', 'o', 'u', # Vocales cortas
    'ay', 'ee', 'igh', 'oh', 'oo', # Vocales largas y diptongos
    'b',"c", 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'w', 'y', 'z',
    'ch', 'sh', 'th', 'ng'
]

print(f"Se generarán {len(phonemes_es)} audios para español.")
print(f"Se generarán {len(phonemes_en)} audios para inglés.")

Se generarán 27 audios para español.
Se generarán 33 audios para inglés.


## Paso 3: Generación de los Archivos de Audio

Ahora, creamos una función que toma una lista de fonemas, un idioma y una carpeta de salida, y genera los archivos `.wav`. El proceso es: `Texto -> gTTS -> .mp3 temporal -> pydub -> .wav final`.

In [3]:
def generate_audio_files(phoneme_list, lang_code, output_path):
    """
    Genera archivos de audio .wav para una lista de fonemas usando gTTS.
    
    Args:
        phoneme_list (list): Lista de strings a convertir en audio.
        lang_code (str): Código de idioma para gTTS (ej. 'es', 'en').
        output_path (Path): Directorio donde se guardarán los archivos .wav.
    """
    print(f"\n--- Iniciando generación para el idioma: {lang_code} ---")
    
    for i, phoneme in enumerate(phoneme_list):
        # Definir la ruta del archivo de salida
        wav_filepath = output_path / f"{phoneme}.wav"
        
        # Omitir si el archivo ya existe
        if wav_filepath.exists():
            print(f"({i+1}/{len(phoneme_list)}) El archivo para '{phoneme}' ya existe. Omitiendo.")
            continue
            
        try:
            # 1. Usar gTTS para crear el objeto de audio
            # El 'slow=True' ayuda a una pronunciación más clara de sonidos cortos
            tts = gTTS(text=phoneme, lang=lang_code, slow=True)
            
            # 2. Guardar el audio en un archivo .mp3 temporal
            mp3_temp_path = output_path / "temp.mp3"
            tts.save(mp3_temp_path)
            
            # 3. Usar pydub para cargar el .mp3 y exportarlo como .wav
            audio = AudioSegment.from_mp3(mp3_temp_path)
            # Formato WAV: 16-bit PCM, 16kHz sample rate (estándar para habla)
            audio.export(wav_filepath, format="wav", parameters=["-ac", "1", "-ar", "16000"])
            
            # 4. Eliminar el archivo temporal
            os.remove(mp3_temp_path)
            
            print(f"({i+1}/{len(phoneme_list)}) Archivo para '{phoneme}' guardado correctamente.")

        except Exception as e:
            print(f"Error al procesar '{phoneme}': {e}")
            # Si hubo un error, asegurarse de que el temporal se borre
            if os.path.exists(mp3_temp_path):
                os.remove(mp3_temp_path)

# Generar audios para español
generate_audio_files(phonemes_es, 'es', output_dir_es)

# Generar audios para inglés
generate_audio_files(phonemes_en, 'en', output_dir_en)

print("\n--- Proceso de generación de audio completado. ---")


--- Iniciando generación para el idioma: es ---
(1/27) El archivo para 'a' ya existe. Omitiendo.
(2/27) El archivo para 'e' ya existe. Omitiendo.
(3/27) El archivo para 'i' ya existe. Omitiendo.
(4/27) El archivo para 'o' ya existe. Omitiendo.
(5/27) El archivo para 'u' ya existe. Omitiendo.
(6/27) El archivo para 'b' ya existe. Omitiendo.
(7/27) El archivo para 'c' ya existe. Omitiendo.
(8/27) El archivo para 'd' ya existe. Omitiendo.
(9/27) El archivo para 'f' ya existe. Omitiendo.
(10/27) El archivo para 'g' ya existe. Omitiendo.
(11/27) El archivo para 'j' ya existe. Omitiendo.
(12/27) El archivo para 'h' ya existe. Omitiendo.
(13/27) El archivo para 'k' ya existe. Omitiendo.
(14/27) El archivo para 'l' ya existe. Omitiendo.
(15/27) El archivo para 'm' ya existe. Omitiendo.
(16/27) El archivo para 'n' ya existe. Omitiendo.
(17/27) El archivo para 'ñ' ya existe. Omitiendo.
(18/27) El archivo para 'p' ya existe. Omitiendo.
(19/27) El archivo para 'r' ya existe. Omitiendo.
(20/27) El